In [53]:
# Standard libraries
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.rcParams['figure.dpi'] = 300
import random
import csv
import pandas as pd
import h5py
# Scikit learn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# PyTorch libraries
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize 
# Own scripts:
%load_ext autoreload
%autoreload 2
import physics
import data
import nnc2p
# from nnc2p import NeuralNetwork # our own architecture
# Get dirs
import os
cwd = os.getcwd()# "Code" folder
master_dir = os.path.abspath(os.path.join(cwd, ".."))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Point towards the folder where we store the eos tables (__Note:__ they are not in the Github as these are very large files)

In [6]:
eos_tables_folder = os.path.join("D:/Coding/Datasets/eos_tables")
print(f"Going to look for EOS table at {eos_tables_folder}")

Going to look for EOS table at D:/Coding/Datasets/eos_tables


# Introduction

Here, we try to find a way to generalize the NN approach from the first semester to the situation of tabular EOS. More work coming soon!

# Exploring EOS tables

In [7]:
# Put the downloaded EOS tables here
first_table_filename       = "LS180_234r_136t_50y_analmu_20091212_SVNr26.h5"
second_table_filename = "GShen_NL3EOS_rho280_temp180_ye52_version_1.1_20120817.h5"
third_table_filename      = "SLy4_0000_rho391_temp163_ye66.h5"
# Then specify which we are going to use here
eos_table_filename = third_table_filename

Read in the SLy4 EOS table using our py script:

In [9]:
eos_table = physics.read_eos_table(os.path.join(eos_tables_folder, eos_table_filename))
dim_ye, dim_temp, dim_rho = eos_table["pointsye"][()][0], eos_table["pointstemp"][()][0], eos_table["pointsrho"][()][0]
print(f"This EOS table has dimensions {dim_ye} x {dim_temp} x {dim_rho}")

This EOS table has dimensions 66 x 163 x 391


In [10]:
np.shape(eos_table["logenergy"][()])  # same dimension

(66, 163, 391)

In [12]:
# Small test to see the output of the EOS table
test_ye = eos_table["ye"][()][0]
test_temp = eos_table["logtemp"][()][0]
test_rho = eos_table["logrho"][()][0]
test_press = eos_table["logpress"][()][0, 0, 0]
print(f"For ye {test_ye}, log temp {test_temp}, log rho {test_rho}, we have log p: {test_press}.")

For ye 0.005, log temp -3.0, log rho 3.0239960056064277, we have log p: 17.99956975587081.


See what is inside this EOS table

In [96]:
# Iterate over keys and save them to list for simplified viewing
keys = []
for key in eos_table:
    keys.append(key)
print(keys)
print(len(keys))


['Abar', 'Albar', 'MERGE-space.in', 'MERGE-src.tar.gz', 'MERGE-tables.in', 'MERGE-transition.in', 'SNA-skyrme.in', 'SNA-space.in', 'SNA-src.tar.gz', 'Xa', 'Xh', 'Xl', 'Xn', 'Xp', 'Zbar', 'Zlbar', 'cs2', 'dedt', 'dpderho', 'dpdrhoe', 'energy_shift', 'entropy', 'gamma', 'have_rel_cs2', 'logenergy', 'logpress', 'logrho', 'logtemp', 'meffn', 'meffp', 'mu_e', 'mu_n', 'mu_p', 'muhat', 'munu', 'pointsrho', 'pointstemp', 'pointsye', 'r', 'u', 'ye']
41


# Generating training data by sampling from EOS table

To generate new data

In [13]:
# dat = physics.generate_tabular_data(eos_table, number_of_points = 100000, save_name = "SLy4_training_data")
# dat = physics.generate_tabular_data(eos_table, number_of_points = 20000, save_name = "SLy4_test_data")

Load data

In [103]:
df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_training_data.csv"))
df

,rho,eps,v,temp,ye,p,D,S,tau
0,13.590663,19.381445,0.683568,-0.333333,0.045,31.985334,18.620245,580.748110,655.358209
1,6.457329,28.786040,0.041886,2.166667,0.545,34.766248,6.463001,13.269992,186.431092
2,11.323996,19.654200,0.558550,0.866667,0.535,30.418591,13.652070,310.329455,393.570454
3,8.723996,19.246855,0.009527,-0.300000,0.095,26.341137,8.724392,2.749496,167.935288
4,5.523996,29.186040,0.334705,2.033333,0.095,34.232914,5.862105,103.365923,195.482542
...,...,...,...,...,...,...,...,...,...
99995,3.323996,25.652376,0.577186,0.600000,0.435,28.498601,4.070468,125.881369,157.178893
99996,9.757329,19.124371,0.376611,-2.900000,0.295,27.371010,10.532847,140.768515,238.842241
99997,9.590663,19.114310,0.266570,-0.833333,0.305,27.176186,9.950723,90.427775,207.064143
99998,14.690663,20.976110,0.476948,2.233333,0.485,35.391058,16.714229,326.155098,461.688397


The network architecture we will use:

In [ ]:
class Net(nn.Module):
    """
    Implements a simple feedforward neural network.
    """
    def __init__(self, nb_of_inputs: int = 3, nb_of_outputs: int = 1, h: list = [600, 200], reg: bool = False, activation_function = torch.nn.Sigmoid) -> None:
        """
        Initialize the neural network class.
        """
        # Call the super constructor first
        super(Net, self).__init__()
        
        # For convenience, save the sizes of the hidden layers as fields as well
        self.h = h
        # Add visible layers as well: input is 3D and output is 1D
        self.h_augmented = [nb_of_inputs] + h + [nb_of_outputs]

        # Add field to specify whether or not we do regularization
        self.regularization = reg

        # Define the layers:
        for i in range(len(self.h_augmented)-1):
            if i == len(self.h_augmented)-2:
                setattr(self, f"linear{i+1}", nn.Linear(self.h_augmented[i], self.h_augmented[i+1], bias=False))
            else:
                setattr(self, f"linear{i+1}", nn.Linear(self.h_augmented[i], self.h_augmented[i+1]))
                setattr(self, f"activation{i+1}", activation_function())

    def forward(self, x):
        """
        Computes a forward step given the input x.
        :param x: Input for the neural network.
        :return: x: Output neural network
        """

        for i, module in enumerate(self.modules()):
            # The first module is the whole NNC2P object, continue
            if i == 0:
                continue
            x = module(x)

        return x

# First goal: NNEOS

__NNEOS__: try to replicate the EOS table (at least the core variables we are interested in) using the "input" variables rho, temp, ye.

Get the training data as DataSet and DataLoader objects. Note on normalization: we fit transform on the training data, then use the fitted scaler object to transform (i.e. using same transformation as the training data) the test set.

In [119]:
# Give the names of the input vars (features) and output vars (labels)
in_vars = ["rho", "temp", "ye"]
out_vars = ["eps", "p"]
# For normalization, use sklearn's StandardScaler
scaler = StandardScaler()
# Read the sampled data as pandas dataframes
train_df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_training_data.csv"))
test_df  = pd.read_csv(os.path.join(master_dir, "Data/SLy4_test_data.csv"))
# Convert to PyTorch Datasets as we defined them
train_dataset = data.CustomDataset(train_df, feature_names = in_vars, label_names = out_vars, normalization_function = scaler.fit_transform) 
test_dataset  = data.CustomDataset(test_df, feature_names = in_vars, label_names = out_vars, normalization_function = scaler.transform)
# Then create dataloaders, with batch size 32, from datasets
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader  = DataLoader(test_dataset, batch_size=32)

Create a new instance of the Net:

In [120]:
model = Net(nb_of_inputs = len(in_vars), nb_of_outputs = len(out_vars), h=[100, 100, 100])
model

Net(
  (linear1): Linear(in_features=3, out_features=100, bias=True)
  (activation1): Sigmoid()
  (linear2): Linear(in_features=100, out_features=100, bias=True)
  (activation2): Sigmoid()
  (linear3): Linear(in_features=100, out_features=100, bias=True)
  (activation3): Sigmoid()
  (linear4): Linear(in_features=100, out_features=2, bias=False)
)

In [121]:
nnc2p.count_parameters(model)

20800

Create a trainer object from it:

In [122]:
trainer = nnc2p.Trainer(model, 1e-2, train_dataloader=train_dataloader, test_dataloader=test_dataloader)

In [123]:
trainer.train()

Training the model for 500 epochs.

 Epoch 0 
 --------------
Train loss: 4.44E-01
Test  loss: 4.45E-01

 Epoch 1 
 --------------
Train loss: 7.15E-03
Test  loss: 7.16E-03

 Epoch 2 
 --------------
Train loss: 1.26E-02
Test  loss: 1.24E-02

 Epoch 3 
 --------------
Train loss: 9.31E-03
Test  loss: 9.44E-03

 Epoch 4 
 --------------
Train loss: 1.58E-02
Test  loss: 1.60E-02

 Epoch 5 
 --------------
Train loss: 1.10E-02
Test  loss: 1.10E-02

 Epoch 6 
 --------------
Train loss: 7.63E-03
Test  loss: 7.65E-03

 Epoch 7 
 --------------
Train loss: 2.66E-03
Test  loss: 2.69E-03

 Epoch 8 
 --------------
Train loss: 8.83E-03
Test  loss: 8.72E-03

 Epoch 9 
 --------------
Train loss: 4.94E-03
Test  loss: 4.94E-03

 Epoch 10 
 --------------
Train loss: 4.90E-03
Test  loss: 4.91E-03

 Epoch 11 
 --------------
Train loss: 5.38E-03
Test  loss: 5.39E-03

 Epoch 12 
 --------------
Adapting learning rate to 0.005
Train loss: 2.22E-02
Test  loss: 2.23E-02

 Epoch 13 
 --------------
Train

Create a quick sketch of training

In [81]:
# plt.plot(trainer.train_losses, color='red', label="Train loss")
# plt.plot(trainer.test_losses, color='blue', label="Test loss")

# plt.grid()
# plt.legend()
# for ind in trainer.adaptation_indices:
#     plt.axvline(ind, ls = '--', color='grey')
# plt.yscale('log')
# plt.xlabel("Epochs")
# plt.xlabel("MSE Loss")
# plt.title("Training (100, 100) network tabular EOS for p and eps")
# plt.savefig("testing_training_tab_eos_network_100_100.pdf", bbox_inches = 'tight')
# plt.show()

# Second goal: NNC2P

__TO DO__ think about design of architecture AND fix the conserved variable values -- I think they were not computed correctly before! 

__NNC2P__: try to replicate the full C2P conversion.

Get the training data as DataSet and DataLoader objects. Note on normalization: we fit transform on the training data, then use the fitted scaler object to transform (i.e. using same transformation as the training data) the test set.

In [ ]:
# Give the names of the input vars (features) and output vars (labels)
in_vars = ["rho", "eps", "ye"]
out_vars = ["temp"]
# For normalization, use sklearn's StandardScaler
scaler = StandardScaler()
# Read the sampled data as pandas dataframes
train_df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_training_data.csv"))
test_df  = pd.read_csv(os.path.join(master_dir, "Data/SLy4_test_data.csv"))
# Convert to PyTorch Datasets as we defined them
train_dataset = data.CustomDataset(train_df, feature_names = in_vars, label_names = out_vars, normalization_function = scaler.fit_transform) 
test_dataset  = data.CustomDataset(test_df, feature_names = in_vars, label_names = out_vars, normalization_function = scaler.transform)
# Then create dataloaders, with batch size 32, from datasets
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader  = DataLoader(test_dataset, batch_size=32)

Create a new instance of the Net:

In [ ]:
model = Net(nb_of_inputs = 3, nb_of_outputs = 1, h=[50, 50])
model

Net(
  (linear1): Linear(in_features=3, out_features=50, bias=True)
  (activation1): Sigmoid()
  (linear2): Linear(in_features=50, out_features=50, bias=True)
  (activation2): Sigmoid()
  (linear3): Linear(in_features=50, out_features=1, bias=False)
)

In [ ]:
nnc2p.count_parameters(model)

2800

Create a trainer object from it:

In [ ]:
trainer = nnc2p.Trainer(model, 1e-1, train_dataloader=train_dataloader, test_dataloader=test_dataloader)

In [ ]:
trainer.train()

Training the model for 500 epochs.

 Epoch 0 
 --------------
Train loss: 7.59E-01
Test  loss: 7.62E-01

 Epoch 1 
 --------------
Train loss: 9.92E-01
Test  loss: 9.98E-01

 Epoch 2 
 --------------
Train loss: 9.45E-01
Test  loss: 9.61E-01

 Epoch 3 
 --------------
Train loss: 8.24E-01
Test  loss: 8.29E-01

 Epoch 4 
 --------------
Train loss: 9.10E-01
Test  loss: 9.16E-01

 Epoch 5 
 --------------
Train loss: 8.80E-01
Test  loss: 8.91E-01

 Epoch 6 
 --------------
Train loss: 8.62E-01
Test  loss: 8.67E-01

 Epoch 7 
 --------------
Train loss: 8.20E-01
Test  loss: 8.27E-01

 Epoch 8 
 --------------
Train loss: 8.80E-01
Test  loss: 8.94E-01

 Epoch 9 
 --------------
Train loss: 9.20E-01
Test  loss: 9.26E-01

 Epoch 10 
 --------------
Train loss: 7.34E-01
Test  loss: 7.39E-01

 Epoch 11 
 --------------
Train loss: 7.48E-01
Test  loss: 7.56E-01

 Epoch 12 
 --------------
Train loss: 7.28E-01
Test  loss: 7.31E-01

 Epoch 13 
 --------------
Train loss: 8.87E-01
Test  loss: 8.99

KeyboardInterrupt: 

Create a quick sketch of training

In [ ]:
# plt.plot(trainer.train_losses, color='red', label="Train loss")
# plt.plot(trainer.test_losses, color='blue', label="Test loss")

# plt.grid()
# plt.legend()
# for ind in trainer.adaptation_indices:
#     plt.axvline(ind, ls = '--', color='grey')
# plt.yscale('log')
# plt.xlabel("Epochs")
# plt.xlabel("MSE Loss")
# plt.title("Training (100, 100) network tabular EOS for p and eps")
# plt.savefig("testing_training_tab_eos_network_100_100.pdf", bbox_inches = 'tight')
# plt.show()

# Archive: NNE2T

__NNE2T__: try to replicate the conversion from energy to temperature, which is currently done by rootfinding approximations & lookups in the EOS table (see Gmunu code). It seemed harder than I initially thought to model and train this; and in the end, I'm not sure how useful it'll be, so I'm archiving this.

Get the training data as DataSet and DataLoader objects. Note on normalization: we fit transform on the training data, then use the fitted scaler object to transform (i.e. using same transformation as the training data) the test set.

In [105]:
# Give the names of the input vars (features) and output vars (labels)
in_vars = ["rho", "eps", "ye"]
out_vars = ["temp"]
# For normalization, use sklearn's StandardScaler
scaler = StandardScaler()
# Read the sampled data as pandas dataframes
train_df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_training_data.csv"))
test_df  = pd.read_csv(os.path.join(master_dir, "Data/SLy4_test_data.csv"))
# Convert to PyTorch Datasets as we defined them
train_dataset = data.CustomDataset(train_df, feature_names = in_vars, label_names = out_vars, normalization_function = scaler.fit_transform) 
test_dataset  = data.CustomDataset(test_df, feature_names = in_vars, label_names = out_vars, normalization_function = scaler.transform)
# Then create dataloaders, with batch size 32, from datasets
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader  = DataLoader(test_dataset, batch_size=32)

Create a new instance of the Net:

In [115]:
model = Net(nb_of_inputs = 3, nb_of_outputs = 1, h=[50, 50])
model

Net(
  (linear1): Linear(in_features=3, out_features=50, bias=True)
  (activation1): Sigmoid()
  (linear2): Linear(in_features=50, out_features=50, bias=True)
  (activation2): Sigmoid()
  (linear3): Linear(in_features=50, out_features=1, bias=False)
)

In [116]:
nnc2p.count_parameters(model)

2800

Create a trainer object from it:

In [117]:
trainer = nnc2p.Trainer(model, 1e-1, train_dataloader=train_dataloader, test_dataloader=test_dataloader)

In [118]:
trainer.train()

Training the model for 500 epochs.

 Epoch 0 
 --------------
Train loss: 7.59E-01
Test  loss: 7.62E-01

 Epoch 1 
 --------------
Train loss: 9.92E-01
Test  loss: 9.98E-01

 Epoch 2 
 --------------
Train loss: 9.45E-01
Test  loss: 9.61E-01

 Epoch 3 
 --------------
Train loss: 8.24E-01
Test  loss: 8.29E-01

 Epoch 4 
 --------------
Train loss: 9.10E-01
Test  loss: 9.16E-01

 Epoch 5 
 --------------
Train loss: 8.80E-01
Test  loss: 8.91E-01

 Epoch 6 
 --------------
Train loss: 8.62E-01
Test  loss: 8.67E-01

 Epoch 7 
 --------------
Train loss: 8.20E-01
Test  loss: 8.27E-01

 Epoch 8 
 --------------
Train loss: 8.80E-01
Test  loss: 8.94E-01

 Epoch 9 
 --------------
Train loss: 9.20E-01
Test  loss: 9.26E-01

 Epoch 10 
 --------------
Train loss: 7.34E-01
Test  loss: 7.39E-01

 Epoch 11 
 --------------
Train loss: 7.48E-01
Test  loss: 7.56E-01

 Epoch 12 
 --------------
Train loss: 7.28E-01
Test  loss: 7.31E-01

 Epoch 13 
 --------------
Train loss: 8.87E-01
Test  loss: 8.99

KeyboardInterrupt: 

Create a quick sketch of training

In [ ]:
# plt.plot(trainer.train_losses, color='red', label="Train loss")
# plt.plot(trainer.test_losses, color='blue', label="Test loss")

# plt.grid()
# plt.legend()
# for ind in trainer.adaptation_indices:
#     plt.axvline(ind, ls = '--', color='grey')
# plt.yscale('log')
# plt.xlabel("Epochs")
# plt.xlabel("MSE Loss")
# plt.title("Training (100, 100) network tabular EOS for p and eps")
# plt.savefig("testing_training_tab_eos_network_100_100.pdf", bbox_inches = 'tight')
# plt.show()